In [2]:
submission_rate = [20, 40, 60, 80, 100, 200, 400, 600, 800, 1000, 2000, 4000, 6000, 8000, 10000]
from pathlib import Path
import os

for i in range(len(submission_rate)):
    filename = "submit_rate_intra_{}.log".format(submission_rate[i])
    grepped_files_prefix = f"../logs/receiver_submit_rate_{submission_rate[i]}*"
    
    # Use os.system to execute the grep command
    os.system(f'grep "Forwarding request " {grepped_files_prefix} > {filename}')
    print("filename:", filename)

filename: submit_rate_intra_2000.log
filename: submit_rate_intra_4000.log
filename: submit_rate_intra_6000.log
filename: submit_rate_intra_8000.log
filename: submit_rate_intra_10000.log


In [3]:
import re
def parse_int_tag(line, name):
    match = re.search(f"{name}=([0-9]*)", line)
    if match:
        return int(match.group(1))
    else:
        return 0

def parse_line(line):
    deadline = parse_int_tag(line, "deadline")
    now = parse_int_tag(line, "now")
    id = parse_int_tag(line, "r_id")
    proxy_id = parse_int_tag(line, "c_id")
    proxy_seq = parse_int_tag(line, "c_seq")

    return (id, proxy_id, proxy_seq)    

In [4]:
from collections import defaultdict

# Get edit distance between 2 sequences from receivers
def req_eq(r1, r2):
    return r1[0] == r2[0] and r1[1] == r2[1]

def edit_dist(s1, s2):
    dp = [ ([-1] * (len(s2) + 1)) for _ in range(len(s1) + 1)]

    dp[0][0] = 0

    for i in range(1, len(dp)):
        dp[i][0] = i
        
    for j in range(1, len(dp[0])):
        dp[0][j] = j
        
    for i in range(1, len(dp)):
        for j in range(1, len(dp[i])):
            # Change req at position i/j
            if req_eq(s1[i-1], s2[j-1]):
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = 1 + dp[i - 1][j - 1]

            # TODO swaps?
            dp[i][j] = min(dp[i][j], 1 + dp[i - 1][j], 1 + dp[i][j - 1])
    
    return dp[-1][-1]


# Longest Increasing Subsequence
def lis(s1, s2, truncation = None):
    # use s1 as truth
    if truncation:
        s1 = s1[:truncation]
        s2 = s2[:truncation]
    
    entry_to_idx = defaultdict(int)
    for i, x in enumerate(s1):
        x = (x[0], x[1])
        entry_to_idx[x] = i

    seq = [entry_to_idx[(x[0], x[1])] for x in s2]    

    dp = []

    for i in range(len(seq)):
        dp.append(1)
        for j in range(i):
            if seq[i] > seq[j]: 
                dp[i] = max(dp[j] + 1, dp[i])

    return max(dp)

def lis_multiple(*seqs):
    s_true = seqs[0]
    n = len(s_true)

    ret = []
    for s in seqs[1:]:
        ret.append(lis(s_true, s) / n)

    return ret

# Simpler metric, which perecentage of requests are the same?
def matching_reqs(*seqs):
    n = len(seqs[0])
    ret = []
    
    for entries in zip(*seqs):
        match = True
        for (x, y) in zip(entries, entries[1:]):
            if not req_eq(x, y): match = False

        ret.append(match)
        
    return ret

In [5]:
from collections import defaultdict
import bisect

def efficient_lis(s1, s2, truncation=None):
    # Use s1 as the ground truth
    if truncation:
        s1 = s1[:truncation]
        s2 = s2[:truncation]
    
    entry_to_idx = defaultdict(int)
    
    for i, x in enumerate(s1):
        x = (x[0], x[1])
        entry_to_idx[x] = i
    
    # Map entries in s2 to their indices in s1
    seq = [entry_to_idx[(x[0], x[1])] for x in s2 if (x[0], x[1]) in entry_to_idx]
    
    # Now compute LIS on the mapped sequence using binary search for efficiency
    lis = []
    
    for num in seq:
        pos = bisect.bisect_left(lis, num)
        if pos == len(lis):
            lis.append(num)
        else:
            lis[pos] = num
    
    return len(lis)

In [9]:
import itertools


requests={}
for i in range(len(submission_rate)):
    requests = {}
    
    filename = "submit_rate_intra_{}.log".format(submission_rate[i])
    print("filename is: ", filename)
    for line in open(filename):
        
        (id, *req) = parse_line(line)

        if id not in requests:
            requests[id] = []
        
        requests[id].append(tuple(req))
        
    # print(requests.keys())
    print(len(requests[1]))
        
    reqs = matching_reqs(*[v for v in requests.values()])
    
    print(f"Total number of requests: {len(reqs)}")
    print(f"Total proportion of requests the same: {sum(reqs) / len(reqs):.2f}")

    # Get runs of smae requests len 10 or above
    # print([(k, sum(1 for _ in g)) for k, g in itertools.groupby(reqs)])

filename is:  submit_rate_intra_2000.log
79879
Total number of requests: 79879
Total proportion of requests the same: 1.00
filename is:  submit_rate_intra_4000.log
148019
Total number of requests: 146150
Total proportion of requests the same: 0.07
filename is:  submit_rate_intra_6000.log
136516
Total number of requests: 136516
Total proportion of requests the same: 0.10
filename is:  submit_rate_intra_8000.log
150666
Total number of requests: 142976
Total proportion of requests the same: 0.01
filename is:  submit_rate_intra_10000.log
142793
Total number of requests: 132636
Total proportion of requests the same: 0.00


In [7]:
lis(requests[0], requests[1], truncation=20000)

12761

In [8]:
for i in range(len(submission_rate)):
    requests = {}
    
    filename = "submit_rate_intra_{}.log".format(submission_rate[i])
    print("filename is: ", filename)
    for line in open(filename):
        
        (id, *req) = parse_line(line)

        if id not in requests:
            requests[id] = []
        
        requests[id].append(tuple(req))
    
    total_len = len(requests[0])
    print("total len: ", total_len)
    print(efficient_lis(requests[0], requests[1]))

filename is:  submit_rate_intra_2000.log
total len:  79879
79691
filename is:  submit_rate_intra_4000.log
total len:  146150
136321
filename is:  submit_rate_intra_6000.log
total len:  141611
87461
filename is:  submit_rate_intra_8000.log
total len:  142976
77578
filename is:  submit_rate_intra_10000.log
total len:  132636
58526
